# How to solve multivariate polynomial systems of equations in Oscar

- Author: Christian Eder
- Version: OSCAR version 1.4.0 or newer.


## Introduction

This tutorial provides an introduction to solving systems of multivariate polynomials in *OSCAR*. It highlights key functionalities through examples, making it easier for users to get started. However, this tutorial does not cover all available functionality for solving over the rationals.

No prior knowledge of *OSCAR* is required for this tutorial, but a basic understanding of Julia is assumed.

## Contents

The content of this tutorial is as follows:

1. [The Concept of Algebraic Solving](#1-The-Concept-of-Algebraic-Solving)  
2. [Aspects of Efficiency for Computing a Representation of the Solution Set](#2-Aspects-of-Efficiency-for-Computing-a-Representation-of-the-Solution-Set)  
3. [Finitely Many Solutions](#3-Finitely-Many-Solutions)  
4. [Infinitely Many Solutions](#4-Infinitely-Many-Solutions)  
5. [## 5 High-level OSCAR Functionality for Solving Multivariate Polynomial Systems with Finitely Many Solutions](#5-High-level-OSCAR-Functionality-for-Solving-Multivariate-Polynomial-Systems-with-Finitely-Many-Solutions)  
6. [Matrix Spaces](#6-Matrix-Spaces)  

In [ ]:
using Oscar

In [ ]:
Oscar.versioninfo()

## 1 The concept of algebraic solving

Solving a system of multivariate polynomials in variables $x_1,...,x_n$ means to find values for the variables such that all polynomials of the system _simultaneously_ evaluate to zero. Now, solving algebraically is based on the computation of a Gröbner basis w.r.t. an elimination ordering, mostly the _lexicographical_ ordering eliminating every variable, say $G_{lex}$. 

Let us define a polynomial ring in two variables $x$ and $y$ over the rationals in OSCAR.

In [ ]:
R, (x,y) = polynomial_ring(QQ, ["x","y"]);

We define the system of multivariate polynomials for which we want to find solutions as an ideal in $R$.

In [ ]:
I = ideal(R, [x^2+-y, x*y^2 -y]);

Next we compute a Gröbner basis for $I$ w.r.t. the lexicographical monomial ordering.

In [ ]:
groebner_basis(I, ordering=lex(R))

If the input system is zero dimensional, i.e. if there exist only finitely many solutions, then there exists a univariate polyniomial $f(x_n)$ which one can solve ``easily''.

In our example this univariate polynomial is $y^4-y = (y^3-1)*y$ and has the solutions $0$ and $1$. Next, propagating the solutions for $y$ into the other polynomials of the lexicographical Gröbner basis we receive the set of solutions for $I$, namely $\{(0,0), (1,1)\}$.

## 2 Aspects of Efficiency for Computing a Representation of the Solution Set

In general, directly computing a Gröbner basis of the input system _w.r.t. the lexicographical ordering_ is not efficient. Here, the concept of _Gröbner conversion_ is helpful. Instead of going the direct way, one uses an indirect approach based on two main steps:
1. Compute a Gröbner basis w.r.t. an efficient monomial ordering $<_1$, say $G_{<_1}$.
2. Convert $G_{<_1}$ to a Gröbner basis $G_{lex}$ w.r.t. the lexicographical ordering.

For conversion there exists two main attempts, depending on the number of solutions of the input system.

## 3 Finitely Many Solutions

In the situation of finitely many solutions one of the most efficient attempts is to call the FGLM algorithm once $G_{<_1}$ is computed. FGLM is based on linear algebra only and converts $G_{<_1}$ efficiently to $G_{lex}$ without the need of recomputing a Gröbner basis.

In [ ]:
G = groebner_basis(I, ordering=degrevlex(R), complete_reduction=true)

In [ ]:
Oscar._fglm(G, lex(R))

As can be seen from the OSCAR code above, there are some minor pitfalls, e.g. the input Gröbner basis for FGLM needs to be reduced. Thus, instead of applying the internal function `Oscar._fglm` we have user facing function called `fglm` which does both of the above steps at once:

In [ ]:
fglm(I, start_ordering=degrevlex(R), destination_ordering=lex(R))

Here, `start_ordering` should be efficient for the input system. In general, the degree reverse lexicographical ordering is a good choice, but sometimes weighted orderings, etc. are better suited. This strongly depends on the input and its context. If one just wants to use the default ordering of the underlying polynomial ring $R$ (in general, degree reverse lexicographical ordering) as default start ordering, one can also call the FGLM variant via `groebner_basis` directly:

In [ ]:
groebner_basis(I, algorithm=:fglm, ordering=lex(R))

## 4 Infinitely Many Solutions

If the input system is not zero dimensional, but positive dimensional, the FGLM algorithm cannot be applied. Moreover, _solving_ in this setting means to find a description of the solutions from which the relevant properties of the solutions are easy to extract. What kind of description one wants to have depends on the input and its context. Here, we restrict ourselves to the approach of converting a given Gröbner basis, which is most often the basis of finding a description of the solutions.  

The concept of _Gröbner walk_ can be used to convert bases in positive dimension. This concept is in general slower than FGLM since it cannot directly convert a basis $G_{<_1}$ to a basis $G_{<_2}$, but has to run over a path in the so-called _Gröbner cone_ over several different monomial orderings before reaching a Gröbner basis w.r.t. $<_2$.

In [ ]:
groebner_walk(I)

The above OSCAR function takes the ideal `I` of input generators as input. By default, it first computes a Gröbner basis for `I` w.r.t. the default ordering of the underlying polynomial ring $R$ (in general, degree reverse lexicographical ordering) and then converts it to a Gröbner basis w.r.t. the lexicographical ordering. If one wants to adjust these start and target orderings one can specify the corresponding parameters:

In [ ]:
groebner_walk(I, lex(R), degrevlex(R))

*Note* that the above implementation of `groebner_walk()` is still in experimental status, thus things like function call, parameter naming, etc. are due to change in upcoming releases.

## 5 High-level OSCAR Functionality for Solving Multivariate Polynomial Systems with Finitely Many Solutions

Besides computing representations of the solution sets ``by hand'' via applying Gröbner basis commands as described above, OSCAR provides a more high-level approach for solving mutlivariate polynomial systems with finitely many solutions. One can either ask for real solutions (up to a given precision) or for rational solutions. Let us take an example in $R$ which is a bit more complex

In [ ]:
I = ideal(R, [x^2-y-1, x+y -y^2+1])

In [ ]:
sol = real_solutions(I)

First of all, this seems to be a lot of information, but let us have a closer look at `sol`. It consists of two parts: The first one is the set of real solutions:

In [ ]:
sol[1]

The second part represents the solution set as a rational parametrization, one can think of as a condensed version of the Gröbner basis for `I` w.r.t. the lexicographical ordering.

In [ ]:
sol[2]

If one is only interested in the rational solutions of the input systems, one does not need to extract the rational solutions from the real solution set above, but can call directly:

In [ ]:
rational_solutions(I)